In [7]:
%pip install numerapi pandas joblib

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import pandas as pd
from numerapi import CryptoAPI
import joblib

from dotenv import load_dotenv
import os

In [19]:
input_file_path = '../data/crypto_model.pkl'
input_live_universe_file_path = '../data/merged_historical_prices.parquet'

output_live_parquet_file_path = '../data/live_targets.parquet'
output_live_csv_file_path = '../data/live_targets.csv'
output_submission_file_path = '../data/submission.parquet'

In [12]:
# Load the .env file
load_dotenv()

# Access the environment variables
api_key = os.getenv('API_KEY')
api_secret = os.getenv('API_SECRET')

# Use API keys to authenticate
cApi = CryptoAPI(api_key, api_secret)
cApi.download_dataset("crypto/v1.0/live_universe.parquet", dest_path=output_live_parquet_file_path)


2024-08-01 13:29:53,410 WARNING numerapi.base_api: You need to supply both a public id and a secret key.
2024-08-01 13:29:56,248 INFO numerapi.utils: target file already exists
2024-08-01 13:29:56,250 INFO numerapi.utils: download complete


'../data/live_targets.parquet'

In [26]:
historical_data = pd.read_csv('../data/historical_data.csv')
historical_data['date'] = pd.to_datetime(historical_data['date'])

live_df = pd.read_parquet(output_live_parquet_file_path)
live_df.reset_index(drop=True, inplace=True)
live_df.to_csv(output_live_csv_file_path)

# Load the trained model
model = joblib.load(input_file_path)

In [27]:
# Get the latest date available in the historical data
latest_date = historical_data['date'].max()

# Filter historical data for the latest date
latest_data = historical_data[historical_data['date'] == latest_date]

In [32]:

# Merge live data with the latest historical data
live_data = live_df.merge(latest_data, on='symbol', how='left')

# Select the features
feature_cols = ['open', 'high', 'low', 'close', 'volume']

# Ensure live_data has the necessary features
live_features = live_data[feature_cols]

# Predict using the model
live_data["signal"] = model.predict(live_features)

# Ensure predictions are between 0 and 1
live_data["signal"] = live_data["signal"].rank(pct=True)

# Format and save submission
live_data[['symbol', 'signal']].to_parquet(output_submission_file_path)

# Get model ids and submit models
# models = napi.get_models(tournament=12)
# for model_name, model_id in models.items():
#     print(f'submitting {model_name}...')
    # napi.upload_predictions("submission.parquet", model_id=model_id, tournament=12)

print('done!')

done!
